In [ ]:
import numpy as np
import matplotlib.pylab as plt
import pandas as pd

import sys

import grade_tools as gt

import seaborn as sns

import os


# For mailing
import smtplib
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders

import base64
import ssl
import getpass


In [ ]:
infile = 'PHYS_260_Thermal_Physics_S23.csv'

df = pd.read_csv(infile)

In [ ]:
#print(df)

# Drop Enzo
df = df[~(df['Last Name']=='Morina')]

df

In [ ]:
lnames = df['Last Name'].values
fnames = df['First Name'].values

print(lnames)
print(fnames)

In [ ]:
mask = (df['Last Name']=='Cestero') & (df['First Name']=='Anthony')

dft = df[mask]

print(dft)

In [ ]:
for fname,lname in zip(fnames,lnames):
    print(f'{fname} {lname}')
    print(f'{type(fname)} {type(lname)}')

    if type(lname)!=str or type(fname)!=str:
        print("Continuing")
        continue
    
    print(lname,fname)
    mask = (df['Last Name']==lname) & (df['First Name']==fname)
    print(mask)
    dftemp = df[mask]
    print(dftemp)
    print(df[mask])
    print()


In [ ]:
def get_assignments(df):
    
    maps = {'quiz':['QUIZ'], 'hw':['HW', 'HOMEWORK'], "participation":['PARTICIPATION', 'participation']}

    colnames = df.columns

    assignments = {"name":[], "points":[], "date":[], "type":[]}
    
    #assignments['part'] = {'name':[], 'date':[], 'points': []}
    #assignments['quiz'] = {'name':[], 'date':[], 'points': []}
    #assignments['hw'] = {'name':[], 'date':[], 'points': []}
    #assignments['midterm'] = {'name':[], 'date':[], 'points': []}
    #assignments['final'] = {'name':[], 'date':[], 'points': []}
    #assignments['extracredit'] = {'name':[], 'date':[], 'points': []}

    for name in colnames:
        #print(name, " -----")
        for key in maps.keys():
            #print(key)
            MATCH = False
            for val in maps[key]:
                #print(val)
                if name.find(val)>=0:
                    MATCH = True
            #print(MATCH,name,key,val)            
            if MATCH:
                points = df[name].iloc[1]
                #print(type(points))
                if type(points)!=str:
                    continue
                assignments['name'].append(name)
                assignments['type'].append(key)
                assignments['date'].append(df[name].iloc[0])
                assignments['points'].append(float(points))
    
    dfa = pd.DataFrame.from_dict(assignments)
    return dfa
            
'''
            hwandquizinfo[name] = []
        # EDAV
        elif name.find('HW')>=0 or name.find('HOMEWORK')>=0 or name.find('CLASS')>=0:
        # Quantum
        #elif name.find('HW')>=0 or name.find('HOMEWORK')>=0 or name.find('CODING')>=0:
            hwinfo[name] = []
            hwandquizinfo[name] = []
        elif name.find('PARTICIPATION')>=0 or name.find('PARTICIPATION')>=0:
            partinfo[name] = []
        elif name.find('EXTRA')>=0:
            extracreditinfo[name] = []
        elif name.find('MID')>=0:
            midterminfo[name] = []
        elif name.find('FINAL')>=0:
            finalinfo[name] = []
'''

dfa = get_assignments(df)

dfa


In [ ]:
mask = dfa['type']=='quiz'
dfa[mask]

In [ ]:
def represents_float(s):
    try: 
        float(s)
    except ValueError:
        return False
    else:
        return True

In [ ]:
dfa

In [ ]:
names = dfa['name']

dfplot = {"name":[], "scores":[], "type":[]}

for name in names:
    #print(name)
    dftemp = df[name]
    x = dftemp.values[2:] # Skip first two rows which are date and total points
    #print(x[0],represents_float(x[0]))
    if represents_float(x[0]):
        x = x.astype(float)
        #print(x)
        #print(len(x))
        if x[0]==x[0]:

            points = dfa[dfa['name']==name]['points'].values[0]
    
            
            dfplot["name"] += len(x)*[name]
            dfplot['scores'] += (x/points).tolist()
            
            atype = dfa[dfa['name']==name]['type'].values[0]
            dfplot['type'] += len(x)*[atype]

            #plt.figure()
            #plt.hist(x)

dfplot = pd.DataFrame.from_dict(dfplot)

In [ ]:
dfplot

In [ ]:
plt.figure(figsize=(16,4))

sns.violinplot(data=dfplot, y="scores", x="name", hue='type', dodge=False);

plt.xticks(rotation=90);

In [ ]:
plt.figure(figsize=(16,4))

sns.boxplot(data=dfplot, y="scores", x="name", hue='type');

plt.xticks(rotation=90);

In [ ]:
plt.figure(figsize=(16,4))

sns.boxplot(data=dfplot, y="scores", x="name", hue='type', dodge=False);

plt.plot([0],[0.8],'ro',markersize=10);

plt.xticks(rotation=270,ha='left');

In [ ]:
anames = np.unique(dfplot['name'])

plt.figure(figsize=(16,4))

sns.boxplot(data=dfplot, y="scores", x="name", hue='type', dodge=False);

xlabels = plt.gca().get_xticklabels(which='both')

n = 2

xpts = []
ypts = []

output = ""
for idx,xlabel in enumerate(xlabels):
    print(xlabel)
    name = xlabel.get_text()
    print(name)
    org_score = float(df.iloc[n][name])
    points = dfa[dfa['name']==name]['points'].values[0]
    score = org_score/points
    xpts.append(name)
    ypts.append(score)
    
    
lname = df.iloc[n]['Last Name']
fname = df.iloc[n]['First Name']
email = df.iloc[n]['Email Address']

print()

plt.plot(xpts,ypts,'ro-',markersize=10, label=f'{fname} {lname}');

plt.xticks(rotation=270,ha='left');
plt.legend()





In [ ]:
#xlabel.

In [ ]:
df.iloc[2]['Last Name']

In [ ]:
df.iloc[2]

https://medium.com/dunder-data/automatically-wrap-graph-labels-in-matplotlib-and-seaborn-a48740bc9ce

In [ ]:
import textwrap
def wrap_labels(ax, width, break_long_words=False):
    labels = []
    for label in ax.get_xticklabels():
        text = label.get_text()
        labels.append(textwrap.fill(text, width=width,
                      break_long_words=break_long_words))
    ax.set_xticklabels(labels, rotation=270)

In [ ]:
weighting = {"quiz":0.45, "hw": 0.20, "participation":0.05}
weighting_sum = 0
for key in weighting.keys():
    weighting_sum += weighting[key]
    
print(f"Weighting sum: {weighting_sum}")


In [ ]:
dfa

In [ ]:
n = 2

dfstudent = df.iloc[2]

dfstudent

In [ ]:
n = 12

dfstudent = df.iloc[n]

#print(dfstudent)
#print()

nassignments = len(dfa)

scores = {}
for key in weighting.keys():
    scores[key] = []
print(scores)

for n in range(nassignments):
    aname = dfa.iloc[n]['name']
    atype = dfa.iloc[n]['type']
    points = dfa.iloc[n]['points']
    date = dfa.iloc[n]['date']
    
    org_score = float(dfstudent[aname])
    if org_score != org_score:
        continue
        
    score = org_score / points
    scores[atype].append(score)
    
    
print(scores)
dfstudent

In [ ]:
def average_scores(scores, weighting, drop_lowest=False):
    weight_sum = 0
    what_was_included = []
    
    tot = 0
    
    #if drop_lowest:
    #    scores = n
    
    for key in scores:
        if len(scores[key])>0:
            if key not in what_was_included:
                weight_sum += weighting[key]
                what_was_included.append(key)
            ave = np.average(scores[key])
            w = weighting[key]
            
            #print(scores[key], ave, w)

            tot += ave*w
    
    #print(tot, weight_sum)
    tot /= weight_sum
    #print(tot)
    return tot

In [ ]:
average_scores(scores,weighting)

# Reference
https://julien.danjou.info/sending-emails-in-python-tutorial-code-examples/

https://realpython.com/python-send-email/


In [ ]:
################################################################################
def write_and_send_email(sender_email, rec_email, title, name, body_text, subject, attachment=None):

    message = MIMEMultipart("alternative")
    message["Subject"] = subject
    message["From"] = sender_email
    message["To"] = rec_email

    message["Cc"] = "mbellis@siena.edu"

    # For HBCUs/HSIs
    text = ""
    text += body_text

    part1 = MIMEText(text, "plain")

    message.attach(part1)
    
    # Attachment
    # https://stackoverflow.com/questions/69901601/send-email-with-attachment-using-python-3-7-3
    if attachment is not None:
        filename = attachment
        with open(filename, "rb") as fs:
            part = MIMEBase('application', "octet-stream")
            part.set_payload(fs.read())
        encoders.encode_base64(part)
        part.add_header('Content-Disposition', 'attachment', filename=os.path.basename(filename))
        message.attach(part)


    context = ssl.create_default_context()
    
    with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
        server.login(sender_email, password)

        # UNCOMMENT THIS WHEN SENDING FOR REAL
        receiver_email = rec_email

        server.sendmail(sender_email, receiver_email, message.as_string())





In [ ]:
df

In [ ]:
# Make a directory if we need to for the figures
try:
   os.makedirs("figures")
except FileExistsError:
   # directory already exists
   pass


####################################
# For mailing

########################################################################
login = "1a2b3c4d5e6f7g" # paste your login generated by Mailtrap
password = "1a2b3c4d5e6f7g" # paste your password generated by Mailtrap
sender_email = "matthew.bellis@gmail.com"
port = 465
password = getpass.getpass("Your password: ")

###############################################

max_names = 18
#max_names = 3

course_grades = []

for n in range(2,max_names):
#for n in range(16,17):

    plt.figure(figsize=(16,6), dpi=200)

    sns.boxplot(data=dfplot, y="scores", x="name", hue='type', dodge=False);

    xlabels = plt.gca().get_xticklabels(which='both')

    xpts = []
    ypts = []
    
    dfstudent = df.iloc[n]

    scores = {}
    for key in weighting.keys():
        scores[key] = []
    #print(scores)
    
    running_aves = []
    
    lname = dfstudent['Last Name']
    fname = dfstudent['First Name']
    email = dfstudent['Email Address']

    subject = f"PHYS 260 - Thermal Physics: summary grades for {fname} {lname}"
    email_body = f"Summary grades for {fname} {lname} ({email})\n\n"
    
    #email_body += '<html><font="Fixed Width">\n'
    
    for idx,xlabel in enumerate(xlabels):
        #print(xlabel)
        name = xlabel.get_text()
        #print(name)
        org_score = float(dfstudent[name])
        points = dfa[dfa['name']==name]['points'].values[0]
        score = org_score/points
        xpts.append(name)
        ypts.append(score)
        
        #print(xlabel)
        atype = dfa[dfa['name']==xlabel.get_text()]['type'].values[0]
        date = dfa[dfa['name']==xlabel.get_text()]['date'].values[0]

        #print(atype,date)
        scores[atype].append(score)
        running_ave = average_scores(scores,weighting)
        running_aves.append(running_ave)
        
        #email_body += f"{name:30s} {atype:10s} {date:15s} {100*score:5.2f} ({org_score}/{points})\n"   
        email_body += f"{100*score:5.2f} ({org_score:5.1f}/{points:5.1f})  {atype:10s} {date:15s} {name:30s} \n"   

        
    #print(f"{fname} {lname} {email}")
    #print(output)
    #email_body += f"{aname} {atype} {points} {date} {score}\n")
    
    course_grade = running_aves[-1]
    
    course_grades.append(course_grade)
    
    email_body += f"\nCourse grade: {100*course_grade:8.2f}\n"

    #email_body += '</font></html>'

    
    print(email_body)

    
    plt.plot(xpts,ypts,'ro--',markersize=10, linewidth=5, label=f'{lname}: Assignment grades');
    plt.plot(xpts,running_aves,'ko--',markersize=10, linewidth=5, label=f'{lname}: Course grade {100*course_grade:0.1f}');


    #plt.xticks(rotation=270,ha='center',fontsize=14);
    #plt.xticks(rotation=90,ha='center',fontsize=14);
    plt.legend()
    wrap_labels(plt.gca(), 16)
    plt.xlabel('')
    plt.ylabel('Assignment grade',fontsize=14)
    plt.tight_layout()
    
    outfilename = f"figures/PHYS_260_summary_{fname}_{lname}.png"
    plt.savefig(outfilename)

    rec_email = email
    # For testing
    #rec_email = 'mbellis@siena.edu'
    #write_and_send_email(sender_email=sender_email, rec_email=rec_email, title=None, name=None, \
    #                     subject=subject, body_text=email_body, attachment=outfilename)

In [ ]:
scores = {}
for key in weighting.keys():
    scores[key] = []
print(scores)

nassignments = len(dfa)

for n in range(nassignments):
    aname = dfa.iloc[n]['name']
    atype = dfa.iloc[n]['type']
    points = dfa.iloc[n]['points']
    date = dfa.iloc[n]['date']
    
    
    org_score = float(dfstudent[aname])
    if org_score != org_score:
        continue
        
    score = org_score / points
    print(f"{aname} {atype} {points} {date} {score}")
    
    scores[atype].append(score)
    x = average_scores(scores,weighting)
    print(f"Running average: {100*x:.2f}")
    
print(scores)
    

In [ ]:
x = np.sort(100*np.array(course_grades))

for i in x:
    print(i)